In [8]:
import os
import psycopg2
from dotenv import load_dotenv
from pgvector.psycopg2 import register_vector
from langchain.schema.document import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from Embeddings import get_embedding_function
from sqlmodel import Field, Session, SQLModel, create_engine, func, select
from sqlalchemy import Column, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

In [40]:
#set up pg vector DB
load_dotenv(override=True)
DBUSER = os.environ["DBUSER"]
DBPASS = os.environ["DBPASS"]
DBHOST = os.environ["DBHOST"]
DBNAME = os.environ["DBNAME"]
DATABASE_URI = f"postgresql://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}"

# Use SSL if not connecting to localhost
DBSSL = "disable"
if DBHOST != "localhost":
    DBSSL = "require"

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
    host=DBHOST,                   
    database=DBNAME,           
    user=DBUSER,                   
    password=DBPASS,             
    port=5432                     
)
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
conn.commit()    

In [41]:
# Register the vector type with psycopg2
register_vector(conn)

In [42]:
def create_table():
    # Create table to store embeddings and metadata
    table_create_command = """
    DROP TABLE IF EXISTS embeddings;
    CREATE TABLE embeddings (
                id serial primary key, 
                embedding vector(3)
                );
    commit;
                """
    cur.execute(table_create_command)

    print("Table created!")

In [43]:
create_table()

Table created!


In [39]:
r = cur.execute("select * from embeddings;")
print(r)

None


In [48]:
list_vec = [[1,5,7],[3,5,1]]

In [51]:
insert_query = """
            insert into embeddings(embedding)
values('[2,6,3]');
        """

cur.execute(insert_query)
conn.commit()

In [52]:
try:    
    select_query = """
    SELECT id, embedding
    FROM embeddings;
    """

    # Execute the query
    cur.execute(select_query)

    # Fetch all rows from the table
    rows = cur.fetchall()

    # Print the results
    print("Data from the vector_store table:")
    for row in rows:
        print(f"ID: {row[0]}")
        print(f"Embedding: {row[1][:10]}...")  

except psycopg2.DatabaseError as error:
    print(f"Error occurred: {error}")

Data from the vector_store table:
ID: 1
Embedding: [ 0.048023    0.06539188 -0.18712297 -0.02319005  0.04506547  0.03031606
  0.03073655 -0.06787408  0.00549885  0.00029228]...
ID: 2
Embedding: [ 0.04113541  0.0786613  -0.19948198  0.01611731  0.08208283  0.05408793
  0.0431351  -0.02864091  0.00612196  0.00222204]...
ID: 3
Embedding: [ 0.0414041   0.09018993 -0.20486927 -0.01810638  0.01800477  0.02448743
  0.05036951 -0.05032595  0.00432867  0.03186858]...
ID: 4
Embedding: [ 0.04961009  0.0593463  -0.20641    -0.03407237  0.01691355  0.06272545
  0.03745529 -0.01649638  0.01508154 -0.03039037]...
ID: 5
Embedding: [ 0.05275087  0.02933043 -0.19473648 -0.0468386   0.04248914  0.08002943
  0.02536343 -0.02733127  0.02145412 -0.05200835]...
ID: 6
Embedding: [ 0.03739091  0.08101483 -0.19375561 -0.07044255  0.01875902  0.04656063
  0.01856178 -0.01851147 -0.00861324 -0.02222742]...
ID: 7
Embedding: [ 0.04639337  0.08478453 -0.20088236 -0.04738745  0.06132583  0.03021316
  0.02775099 -0.05